# The scenario
The housing market is one of the most crucial parts of the economy for every country. Purchasing a home is one of the primary ways to build wealth and savings for people. In this respect, predicting prices in the housing market is a very central topic in economic and financial circles.

The house price dataset from Kaggle includes several features of the houses along with their sale prices at the time they are sold. So far, in this module, you built and implemented some models using this dataset.

In this challenge, you are required to improve your model with respect to its prediction performance.

To complete this challenge, submit a Jupyter notebook containing your solutions to the following tasks.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse

import warnings
warnings.filterwarnings('ignore')

In [42]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
house_price_df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


house_price_df.describe(include='all')

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
count,1460.000000,1460.000000,1460,1201.000000,1460.000000,1460,91,1460,1460,1460,...,1460.000000,7,281,54,1460.000000,1460.000000,1460.000000,1460,1460,1460.000000
unique,NaN,NaN,5,NaN,NaN,2,2,4,4,2,...,NaN,3,4,4,NaN,NaN,NaN,9,6,NaN
top,NaN,NaN,RL,NaN,NaN,Pave,Grvl,Reg,Lvl,AllPub,...,NaN,Gd,MnPrv,Shed,NaN,NaN,NaN,WD,Normal,NaN
freq,NaN,NaN,1151,NaN,NaN,1454,50,925,1311,1459,...,NaN,3,157,49,NaN,NaN,NaN,1267,1198,NaN
mean,730.500000,56.897260,NaN,70.049958,10516.828082,NaN,NaN,NaN,NaN,NaN,...,2.758904,NaN,NaN,NaN,43.489041,6.321918,2007.815753,NaN,NaN,180921.195890
std,421.610009,42.300571,NaN,24.284752,9981.264932,NaN,NaN,NaN,NaN,NaN,...,40.177307,NaN,NaN,NaN,496.123024,2.703626,1.328095,NaN,NaN,79442.502883
min,1.000000,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,1.000000,2006.000000,NaN,NaN,34900.000000
25%,365.750000,20.000000,NaN,59.000000,7553.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,5.000000,2007.000000,NaN,NaN,129975.000000
50%,730.500000,50.000000,NaN,69.000000,9478.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,6.000000,2008.000000,NaN,NaN,163000.000000
75%,1095.250000,70.000000,NaN,80.000000,11601.500000,NaN,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,NaN,0.000000,8.000000,2009.000000,NaN,NaN,214000.000000


# <font color='red'>Bring in data handling from previos doc</font>

- Include interest rate date for those years into the model
- Create total living area feature
- Create overall quality feature
- Create overall property size feature
- Find financial data to reflect economical activity (customer ability to purchase house and investors likelihood to invest in real estate)


In [76]:
# import interest data and merge with house sale year and month
mortgage_file_path = '/Users/wesamazaizeh/Desktop/Thinkful_Main_Course/Linear_Regression/30-year-fixed-mortgage-rate-chart.csv'
mortgage_apr_df = pd.read_csv(mortgage_file_path, skiprows=15)

mortgage_apr_df['date'] = pd.to_datetime( mortgage_apr_df.date, format='%Y-%m-%d')
mortgage_apr_df['year'] = mortgage_apr_df.date.dt.year
mortgage_apr_df['month'] = mortgage_apr_df.date.dt.month

house_price_df2 = pd.merge(house_price_df, mortgage_apr_df, left_on=['yrsold','mosold'], right_on=['year', 'month'], how='right').drop(['date', 'year', 'month'], axis=1)
house_price_df2.rename(columns={' value':'interest'}, inplace=True)

In [77]:
# Feature engineering

## Total living area
house_price_df2['totalsf'] = house_price_df2.totalbsmtsf + house_price_df2.firstflrsf + house_price_df2.secondflrsf


,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice,interest
0,1.0,60.0,RL,65.0,8450.0,Pave,None,Reg,Lvl,AllPub,...,None,None,None,0.0,2.0,2008.0,WD,Normal,208500.0,6.24
1,11.0,20.0,RL,70.0,11200.0,Pave,None,Reg,Lvl,AllPub,...,None,None,None,0.0,2.0,2008.0,WD,Normal,129500.0,6.24
2,202.0,20.0,RL,75.0,10125.0,Pave,None,Reg,Lvl,AllPub,...,None,MnPrv,None,0.0,2.0,2008.0,WD,Normal,171500.0,6.24
3,269.0,30.0,RM,71.0,6900.0,Pave,None,Reg,Lvl,AllPub,...,None,None,None,0.0,2.0,2008.0,WD,Normal,120500.0,6.24
4,423.0,20.0,RL,100.0,21750.0,Pave,None,Reg,HLS,AllPub,...,None,None,None,0.0,2.0,2008.0,WD,Normal,113000.0,6.24
